<a href="https://colab.research.google.com/github/pikey-msc/RiesgosFinancieros/blob/master/2026-1/SH_Integracion_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# %load_ext rpy2.ipython
!rm -rf RiesgosFinancieros
!git clone "https://github.com/pikey-msc/RiesgosFinancieros/"

Cloning into 'RiesgosFinancieros'...
remote: Enumerating objects: 1234, done.
remote: Counting objects: 100% (466/466), done.
remote: Compressing objects: 100% (288/288), done.
remote: Total 1234 (delta 273), reused 279 (delta 158), pack-reused 768 (from 1)
Receiving objects: 100% (1234/1234), 64.26 MiB | 22.07 MiB/s, done.
Resolving deltas: 100% (778/778), done.
Updating files: 100% (455/455), done.


#Funciones auxiliares

In [4]:
import numpy as np

def equantile(v, p=0.5):
    """
    Calcula el cuantil más cercano de un vector.

    Args:
        v (list or numpy array): La lista o arreglo de números.
        p (float): El percentil deseado (0 <= p <= 1).

    Returns:
        float: El valor del cuantil más cercano.
    """
    if not isinstance(v, (list, np.ndarray)):
        raise ValueError("v debe ser una lista o un arreglo numpy")
    if not (0 <= p <= 1):
        raise ValueError("El percentil debe estar en el rango [0, 1]")

    sorted_v = sorted(v)
    n = len(sorted_v)
    plist = np.cumsum(np.arange(1, n+1) / n)

    return sorted_v[np.argmax(plist >= p)]


# Ejemplo de uso
#v = [4, 7, 1, 9, 3, 5, 8, 2, 6]
#p = 0.6
#resultado = equantile(v, p)
#print(f"El cuantil más cercano al {p*100}% de v es: {resultado}")

import numpy as np

def wquantile(v, w0, p=0.5):
    """
    Calcula el cuantil ponderado de un vector.

    Args:
        v (list or numpy array): La lista o arreglo de números.
        w (list or numpy array): Los pesos correspondientes a los valores en v.
            Si no se proporciona, se asumen pesos iguales.
        p (float): El percentil deseado (0 <= p <= 1).

    Returns:
        float: El valor del cuantil ponderado.
    """
    n=len(v)
    if not isinstance(v, (list, np.ndarray)):
        raise ValueError("v debe ser una lista o un arreglo numpy")

    if w0 is None:
        w = np.ones(len(v))
    else:
        w=[w0 * ((1-w0) ** (i-1)) for i in range(1,n+1)]

    if not isinstance(w, (list, np.ndarray)):
        raise ValueError("w debe ser una lista o un arreglo numpy")
    if len(v) != len(w):
        raise ValueError("Los valores y los pesos deben tener la misma longitud")

    if not (0 <= p <= 1):
        raise ValueError("El percentil debe estar en el rango [0, 1]")

    if min(w) < 0:
        raise ValueError("Los pesos deben ser mayores que 0")

    sorted_indices = np.argsort(v)
    sorted_v = np.array(v)[sorted_indices]
    sorted_w = np.array(w)[sorted_indices]

    sum_w = np.cumsum(sorted_w)
    plist = sum_w / sum_w[-1]

    return np.interp(p, plist, sorted_v)




# Ejemplo de uso
# v = [4, 7, 1, 9, 3, 5, 8, 2, 6]
# w = [1, 2, 1, 1, 2, 1, 1, 1, 2]
# p = 0.6

# resultado = wquantile(v, 0.05, p)
# print(f"El cuantil ponderado al {p*100}% de v es: {resultado}")

import numpy as np

def wcvar(v, w0=None, p=0.5):
    """
    Calcula el Valor en Riesgo Condicional (CVaR) con alisado.

    Args:
        v (list or numpy array): La lista o arreglo de números.
        w (list or numpy array): Los pesos correspondientes a los valores en v.
            Si no se proporciona, se asumen pesos iguales.
        p (float): El nivel de confianza (0 <= p <= 1).

    Returns:
        float: El valor del CVaR con alisado.
    """
    n=len(v)
    if not isinstance(v, (list, np.ndarray)):
        raise ValueError("v debe ser una lista o un arreglo numpy")

    if w0 is None:
        w = np.ones(len(v))
    else:
        w=[w0 * ((1-w0) ** (i-1)) for i in range(1,n+1)]

    if not isinstance(w, (list, np.ndarray)):
        raise ValueError("w debe ser una lista o un arreglo numpy")
    elif len(v) != len(w):
        raise ValueError("Los valores y los pesos deben tener la misma longitud")
    if not (0 <= p <= 1):
        raise ValueError("El nivel de confianza debe estar en el rango [0, 1]")
    if min(w) < 0:
        raise ValueError("Los pesos deben ser mayores que 0")

    sorted_indices = np.argsort(v)
    sorted_v = np.array(v)[sorted_indices]
    sorted_w = np.array(w)[sorted_indices]

    sum_w = np.cumsum(sorted_w)
    plist = sum_w / sum_w[-1]

    loss = sorted_v[plist < p]
    esc = sorted_w[plist < p]

    return np.sum(loss * esc) / np.sum(esc)


# Ejemplo de uso
#v = [4, 7, 1, 9, 3, 5, 8, 2, 6]
#w = [1, 2, 1, 1, 2, 1, 1, 1, 2]
#p = 0.6

#resultado = wcvar(v, 0.05, p)
#print(f"El CVaR con alisado al {p*100}% de v es: {resultado}")


def generar_probabilidades(n, w):
    """
    Genera probabilidades según la fórmula p = w * λ^i.

    Args:
        n (int): Número de probabilidades a generar.
        w (float): Valor de w.

    Returns:
        list: Lista de probabilidades generadas.
    """
    if n <= 0 or w <= 0:
        raise ValueError("n y w deben ser valores positivos")

    lambda_val = 1-w
    probabilidades = [w * ((1-w) ** (i-1)) for i in range(1,n+1)]

    return probabilidades


# Ejemplo de uso
n = 200
w = 0.05

probabilidades_generadas = generar_probabilidades(n, w)
print(f"Probabilidades generadas: {probabilidades_generadas}")
print(sum(probabilidades_generadas))
print(len(probabilidades_generadas))

Probabilidades generadas: [0.05, 0.0475, 0.045125, 0.04286875, 0.0407253125, 0.038689046874999994, 0.03675459453124999, 0.03491686480468749, 0.03317102156445311, 0.03151247048623045, 0.029936846961918936, 0.028440004613822983, 0.027018004383131834, 0.025667104163975243, 0.02438374895577648, 0.023164561507987652, 0.02200633343258827, 0.020906016760958854, 0.019860715922910912, 0.018867680126765363, 0.017924296120427095, 0.01702808131440574, 0.01617667724868545, 0.01536784338625118, 0.01459945121693862, 0.013869478656091689, 0.013176004723287102, 0.012517204487122747, 0.011891344262766609, 0.011296777049628278, 0.010731938197146865, 0.01019534128728952, 0.009685574222925044, 0.009201295511778791, 0.008741230736189851, 0.008304169199380358, 0.00788896073941134, 0.007494512702440773, 0.007119787067318734, 0.006763797713952796, 0.006425607828255156, 0.006104327436842399, 0.005799111065000279, 0.0055091555117502645, 0.005233697736162751, 0.004972012849354612, 0.004723412206886882, 0.00448724

# Definición de parámetros para valorar:
Acciones
Bondes D
SWAPS
Futuros
Opciones

## Definición parámetros Python

In [5]:
!pip install yfinance
import yfinance as yf
import numpy as np
from datetime import datetime, timedelta

fval = datetime.strptime("20230310", "%Y%m%d") #Fecha de valoración
itpl = 0 #poner 0 si se quiere interpolación lineal o 1 si se quiere tasa alambrada
alpha = 0.98 #Nivel de confianza para obtener estimaciones de riesgo
w0=0.9
#setwd(direc)

#ACCIONES Y DIVISAS
#Cargar los símbolos de yahoo finance para EQ
Symbols = ["AMXB.MX", "GCARSOA1.MX", "WALMEX.MX"] #tienen que ir en orden alfabético
pos_eq = [-5000,1000,1200] #monto inicial invertido en acciones
#Cargar los símbolos de yahoo finance para FX
SymbolsFX = ["BTC-USD","EURUSD=X","USDMXN=X"] #tienen que ir en orden alfabético
pos_fx = [1.5,700,1500] #monto inicial invertido en divisas
nh = 3660 #días de historia

#CETES
base = "RiesgosFinancieros/2024-1/Tarea/tasa_guber.txt"


# BONDES D
btasadescst="RiesgosFinancieros/2024-1/Tarea/tasa_guber_st.txt"
btasafondeo="RiesgosFinancieros/2024-1/Tarea/tfondeo.txt"
plazos_bdm=np.array([358, 405, 550, 1200, 1800])
plazocupon_bdm=np.array([28, 28, 28, 28, 28])
contratos_bdm=np.array([220, -290, 290, -460, 100])
nominal_bdm=100

# FORWARDS TDC
bext="RiesgosFinancieros/2024-1/Tarea/tasa_libor.txt"
bdom="RiesgosFinancieros/2024-1/Tarea/tasa_fwd.txt"
SymbolsFX_ftdc=['USDMXN=X']
plazos_fwd=np.array([360, 9, 35, 95, 183])
contratos_fwd=np.array([500, -290, 290, -460, 440])
kst_fwd=[20.5, 19.4, 20.6, 20, 20.5]
nominal_fwd=1
yext=1 #si se carga información de yahoo en la fecha definida por fval o SymbolsFX, en caso contrario se utiliza información que se tendrá que cargar de tasas_spot.txt
trlib=1 #1 si la curva libor viene a 182 0 si no.

# FORWARDS DE IPC
#Descontamos con gubernamental
base="RiesgosFinancieros/2024-1/Tarea/tasa_guber.txt"
SymbolsEQ_find=["^MXX"] #tienen que ir en orden alfabético
plazos_fwd_ind=np.array([360, 9, 3500])
contratos_fwd_ind=np.array([5, -2, 5])
kst_fwd_ind=np.array([52000, 49500, 60000])
nominal_fwd_ind=1


#SWAP
btasadesc_sw="RiesgosFinancieros/2024-1/Tarea/tasa_TIIE_SW_OP.txt"
btasacupvar_sw="RiesgosFinancieros/2024-1/Tarea/tasa_DIRS_SW_OP.txt"
tasafija_sw=np.array([0.079,0.075]) #se establece la tasa fija a pagar para cada swap
plazos_sw=np.array([588,360]) #se establece el número de días que vivirá el swap
plazocupon_sw=np.array([28,28]) #se establece el número de días que se pagará cada cupón
contratos_sw=np.array([-1600,1000]) #se establece el número de contratos_sw de cada swap
nominal_sw=np.array([1,1]) #se establece el nominal_sw de cada swap
por_sw=np.array([0,0]) #se establece el porcentaje de cada swap


#OPCIONES
btasadesc_oir = "RiesgosFinancieros/2024-1/Tarea/tasa_TIIE_SW_OP.txt"
btasaspot_oir = "RiesgosFinancieros/2024-1/Tarea/tasa_DIRS_SW_OP.txt"
bvolspot_oir = "RiesgosFinancieros/2024-1/Tarea/tvoltiie_opc.txt"
plazos_oir = np.array([700, 1700, 400, 350, 200])  # T-t
pr_oir = 28  # plazo de referencia
dct_oir = 360  # d_base
cp_oir = np.array([1, 0, 0, 0, 0])  # si es call (cap) o put (floor)
K_oir = np.array([0.07, 0.085, 0.078, 0.085, 0.08])
contratos_oir = np.array([2200, -2900, 2900, -460, 5000])
nominal_oir = 1
cs_oir = 1  # 1 si es continua la tasa 0 si es simple

# Carga de datos

In [6]:
import pandas as pd
#acciones
pos=[len(pos_fx),len(pos_eq)]
start_date = datetime.now().date() - timedelta(days=nh)
# Symbols = ["AMXB.MX", "GCARSOA1.MX", "WALMEX.MX"] #tienen que ir en orden alfabético
pos_eq = [-5000,1000,1200] #monto inicial invertido en acciones
#Cargar los símbolos de yahoo finance para FX
# SymbolsFX = ["BTC-USD","EURUSD=X","USDMXN=X"] #tienen que ir en orden alfabético
symbol=Symbols+SymbolsFX
# symbol = ['AAPL','MSFT','SQ','AMZN','USDMXN=X']
stock_prices = yf.download(symbol,start_date)
stock_prices_EQFX=stock_prices.Close.dropna()
stock_prices_EQFX['EURUSD=X']=stock_prices_EQFX['EURUSD=X']*stock_prices_EQFX['USDMXN=X']
stock_prices_EQFX['BTC-USD']=stock_prices_EQFX['BTC-USD']*stock_prices_EQFX['USDMXN=X']
stock_prices_EQFX=stock_prices_EQFX[symbol]
stock_prices_EQFX=stock_prices_EQFX.reset_index()
stock_prices_EQFX['Date'] = stock_prices_EQFX['Date'].dt.tz_localize(None)
x0=stock_prices_EQFX.tail(1)
# aux2=pd.DataFrame(stock_prices_EQFX.index)


#BONOS M

# Carga de datos - supongamos que los datos se encuentran en un archivo CSV llamado "btasadesc.csv"
# Asegúrate de proporcionar la ruta completa al archivo o que el archivo esté en el directorio actual.
# data1 = pd.read_("btasadesc.txt", header=None)
data = pd.read_csv(base, sep="\t", header=None)

# Obtiene el número de filas y columnas de data1
n, m_gov= data.shape

# Crear un DataFrame con los datos sin la primera fila y la primera columna
x_orig = pd.DataFrame(data.values[1:, 1:len(data.values[0])], dtype=float)
x_orig['Date'] = pd.to_datetime(data.values[1:,0], format='%Y%m%d')

# Obtener nodos
nodos_gov = pd.DataFrame(data.values[0, 1:(len(data.values[0]))], dtype=int)



# data<-read.table(base)
# n<-nrow(data)
# m_gov=ncol(data)
# #x_orig_gov=data.frame(data[2:n,1:m_gov])
# x_orig_gov=as.data.table(mutate(data[2:n,1:m_gov],Date=as.Date(V1,format="%Y%m%d")))
# x_orig_gov=x_orig_gov%>%select(-V1)
# nodos_gov=data.frame(data[1,2:m_gov])

#BONDE D
# Leer datos de tasa_guber.txt
data1 = pd.read_table(base, sep="\t", header=None)
n = data1.shape[0]
m_bd = data1.shape[1]
X_orig = data1.iloc[1:n, 0:m_bd]
X1_orig = pd.DataFrame(data1.iloc[1:n, 1:m_bd], dtype=float)
X1_orig['Date'] = pd.to_datetime(X_orig[0], format='%Y%m%d')
nodos = data1.iloc[0, 1:m_bd]
n -= 1

# Leer datos de tasa_guber_st.txt
data3 = pd.read_table(btasadescst, sep="\t", header=None)
n3 = data3.shape[0]
m3_bd = data3.shape[1]
X3a_orig_bd = data3.iloc[1:n3, 0:m3_bd]
X3_orig_bd = pd.DataFrame(data3.iloc[1:n3, 1:m3_bd], dtype=float)
X3_orig_bd['Date'] = pd.to_datetime(X3a_orig_bd[0], format='%Y%m%d')
nodos3_bd = data3.iloc[0, 1:m3_bd]
n3 -= 1

# # Leer datos de tfondeo.txt
data2 = pd.read_table(btasafondeo, sep="\t", header=None)
n2 = data2.shape[0]
X2_orig = data2.iloc[1:n2, 0:2]
X2_orig_bd = X2_orig.copy()
X2_orig_bd[0] = pd.to_datetime(X2_orig_bd[0], format='%Y%m%d')
X2_orig_bd[1] = X2_orig_bd[1].astype(float)

# # Sucesión de días para tasa fondeo
tfh = pd.date_range(start=min(X2_orig_bd[0]), end=max(X2_orig_bd[0]), freq='D')
tfhd = pd.DataFrame({'fecha': tfh})


# Asegúrate de que ambos DataFrames estén ordenados por la columna de fecha de manera descendente
tfhd = tfhd.sort_values(by='fecha', ascending=True)
X2_orig_bd = X2_orig_bd.sort_values(by=0, ascending=True)  # Asumiendo que la columna 'V1' es ahora la columna 0

# Renombrar columnas para preparar para merge_asof
# tfhd = tfhd.rename(columns={'fecha': 'join_date'})
X2_orig_bd = X2_orig_bd.rename(columns={0: 'fecha'})

# Realizar el rolling join con merge_asof
X2_orig_bd = pd.merge_asof(tfhd, X2_orig_bd, on='fecha', direction='backward')


# Si quieres que el resultado esté en el DataFrame original (tfhd), puedes hacer lo siguiente:
X2_orig_bd = pd.merge(tfhd, X2_orig_bd, on='fecha', how='outer')

X1_orig = X1_orig.rename(columns={0: "fecha"})
# Filtrar tasas de fondeo

tf_act = X2_orig_bd.loc[X2_orig_bd['fecha'] == fval, 1].values[0] / 100
tf_int = X2_orig_bd.loc[(X2_orig_bd['fecha'] <= fval) & (X2_orig_bd['fecha'] >= (fval -  timedelta(int(plazocupon_bdm[0])))), 1] / 100
tf_int=tf_int[::-1]  # se hace este cambio para ordenar de mayor a menor


# Ordenar el DataFrame por la columna de unión
X1_orig = X1_orig.sort_values('Date', ascending=True)

# Agregar la columna join_date al DataFrame X1_orig
X1_orig['join_date'] = X1_orig['Date']

# Ordenar el DataFrame X2a_orig por la columna de unión
X2_orig_bd = X2_orig_bd.sort_values('fecha', ascending=True)

# Hacer el merge (join) de los DataFrames
X2_pr = pd.merge_asof(X1_orig, X2_orig_bd, left_on='Date', right_on='fecha')



#CARGA DE DATOS DE  FORWARDS DE TDC
#datas
#data<-read.table("tasa_tiie.txt")
data1 = pd.read_table(bext,header=None)
data2 = pd.read_table(bdom,header=None)

########minimos para parametrizar
n1 = data1.shape[0]
n2 = data2.shape[0]
m1_ftdc = data1.shape[1]
m2_ftdc = data2.shape[1]
n = min(n1, n2) - 1
###NODOS###
nodos1_ftdc = data1.iloc[0,1:m1_ftdc]
nodos2_ftdc = data2.iloc[0,1:m2_ftdc]
####MATRICES DEL MISMO TAMAÑO MENOS DOLAR

x1_ftdc = pd.DataFrame(data1.iloc[1:n1,1:m1_ftdc], dtype=float)
x1_ftdc['Date'] = pd.to_datetime(data1.iloc[1:n1,0], format="%Y%m%d")
x2_ftdc = data2.iloc[1:n1,1:m2_ftdc]
x2_ftdc['Date'] = pd.to_datetime(data2.iloc[1:n1, 0], format="%Y%m%d")

if yext==1:
  start_date=fval- timedelta(days=nh)
  start_date = datetime.now().date() - timedelta(days=nh)
  X3_ftdc = yf.download(SymbolsFX_ftdc,start_date)
  X3_ftdc=X3_ftdc.Close.dropna()
  X3_ftdc=X3_ftdc.reset_index()
else:
  data3=pd.read_table(bext,header=None)
  n3 = data3.shape[0]
  m3 = data3.shape[1]
  x3_ftdc = pd.DataFrame(data3.iloc[1:n3,1:m3], dtype=float)
  x3_ftdc['Date'] = pd.to_datetime(data3.iloc[1:n3, 0], format="%Y%m%d")


# CARGA DE DATOS DE  FORWARD DE IPC
data = pd.read_table(base,header=None)
n = data.shape[0]
m_gov=data.shape[1]
#x_orig_gov=data.frame(data[2:n,1:m_gov])
x_orig_gov= pd.DataFrame(data.iloc[1:n,1:m_gov], dtype=float)
x_orig_gov['Date'] = pd.to_datetime(data.iloc[1:n,0], format="%Y%m%d")
nodos_gov=data.iloc[0,1:m_gov]

X3_find = yf.download(SymbolsEQ_find,start_date)
X3_find = X3_find.Close.dropna()
X3_find=X3_find.reset_index()



#CARGA DE DATOS PARA SWAP
data1 = pd.read_table(btasadesc_sw,header=None)
n1 = data1.shape[0]
m1_orig_sw = data1.shape[1]
X1_orig_sw = pd.DataFrame(data1.iloc[1:n1, 1:m1_orig_sw], dtype=float)
X1_orig_sw['Date'] = pd.to_datetime(data1.iloc[1:n1, 0], format = '%Y%m%d')
nodos1_sw = data1.iloc[0, 1:m1_orig_sw]

data2 = pd.read_table(btasacupvar_sw,header=None)
n2 = data2.shape[0]
m2_orig_sw = data2.shape[1]
X2_orig_sw = pd.DataFrame(data2.iloc[1:n2, 1:m2_orig_sw], dtype=float)
X2_orig_sw['Date'] = pd.to_datetime(data2.iloc[1:n1, 0], format = '%Y%m%d')
nodos2_sw = data2.iloc[0, 1:m2_orig_sw]


#carga de opciones de tasa de interés
data1 = pd.read_table(btasadesc_oir,header=None)
n =data1.shape[0]
m1_orig_oir = len(data1.columns)
x1_orig_oir = pd.DataFrame(data1.iloc[1:n, 1:m1_orig_oir], dtype=float)
x1_orig_oir['Date'] = pd.to_datetime(data1.iloc[1:n, 0], format = '%Y%m%d')
nodos1_oir =  data1.iloc[0, 1:m1_orig_oir]

# carga de tasas spot
data2 = pd.read_table(btasaspot_oir,header=None)
n = len(data2)
m2_orig_oir = len(data2.columns)
x2_orig_oir = pd.DataFrame(data2.iloc[1:n, 1:m2_orig_oir], dtype=float)
x2_orig_oir['Date'] = pd.to_datetime(data2.iloc[1:n, 0], format="%Y%m%d")
nodos2_oir = data2.iloc[0, 1:m2_orig_oir]

# carga de volatilidades de spot
data3 = pd.read_table(bvolspot_oir,header=None)
n = len(data3)
m3_orig_oir = len(data3.columns)
x3_orig_oir = pd.DataFrame(data3.iloc[1:n, 1:m3_orig_oir], dtype=float)
x3_orig_oir['Date'] = pd.to_datetime(data3.iloc[1:n, 0], format="%Y%m%d")
nodos3_oir = data3.iloc[0, 1:m3_orig_oir]

/tmp/ipython-input-1413384206.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock_prices = yf.download(symbol,start_date)
[*********************100%***********************]  6 of 6 completed
/tmp/ipython-input-1413384206.py:145: FutureWarning: YF.download() has changed argument auto_adjust default to True
  X3_ftdc = yf.download(SymbolsFX_ftdc,start_date)
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-1413384206.py:165: FutureWarning: YF.download() has changed argument auto_adjust default to True
  X3_find = yf.download(SymbolsEQ_find,start_date)
[*********************100%***********************]  1 of 1 completed


# Integración de Insumos

Aseguramiento de que todos los factores de riesgo estén alineados en sus fechas, nos quedamos con la intersección. Posteriormente, filtramos cada histórico factor de riesgo hacia esta intersección de todos.

## Python Integración de Insumos

In [7]:
from numpy.core.fromnumeric import sort
from numpy.core.fromnumeric import sort
import pandas as pd

def filter_on_date(base_df, join_df, description):
    # Asegurarse de que ambos objetos sean DataFrames con una columna 'Date'
    if not isinstance(base_df, pd.DataFrame):
        base_df = pd.DataFrame({'Date': base_df})
    if not isinstance(join_df, pd.DataFrame):
        join_df = pd.DataFrame({'Date': join_df})

    result = pd.merge(base_df, join_df, on='Date', how='inner')[['Date']].drop_duplicates()
    print(f"Number of rows in {description}: {len(result)}")
    return result

# Inicializar DataFrame con la Fecha desde aux2 y x_orig_gov
# current_df = filter_on_date(aux2, x_orig_gov, "lin_gub")

# Lista de DataFrames para hacer join y sus descripciones
dataframes_to_join = [
    (x_orig_gov, "lin_gub"),
    (X3_orig_bd, "lin_gub_bmybdst"),
    (X3_ftdc, "lin_gub_bmybdst"),
    (x1_ftdc, "lin_gub_bmybdst_flib"),
    (x2_ftdc, "lin_gub_bmybdst_flibfwd"),
    (X3_find, "lin_gub_bmybdst_flibfwdspind"),
    (X1_orig_sw, "lin_gub_bmybdst_flibfwdspind_swcup"),
    (X2_orig_sw, "lin_gub_bmybdst_flibfwdspind_swcupvp"),
    (x1_orig_oir, "lin_gub_bmybdst_flibfwdspind_swcupvp_oirs"),
    (x2_orig_oir, "lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvp"),
    (x3_orig_oir, "lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol")
]

# Realizar los joins y actualizar el DataFrame actual
current_df = stock_prices_EQFX
for join_df, description in dataframes_to_join:
    current_df = filter_on_date(current_df, join_df, description)

# Mostrar el DataFrame final
print(current_df.drop_duplicates())

import pandas as pd

# Suponemos que lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol y otras DataFrames ya están definidas

# Calcula el número de filas
n = len(current_df)

current_df=current_df.sort_values(by="Date", ascending=False)

def join_and_drop_date(base_df, join_df, divide_by_100=False):
    result = pd.merge(base_df, join_df, on='Date', how='inner').sort_values('Date', ascending=False)
    result.drop(columns=['Date'], inplace=True)
    if divide_by_100:
        result = result / 100
    return result

# Historia de acciones y divisas
stock_prices_EQFX = join_and_drop_date(current_df, stock_prices_EQFX)

# Historia de curva gubernamental
x_orig_gov = join_and_drop_date(current_df, x_orig_gov)

# Historia de curvas de bonde
X1_orig = join_and_drop_date(current_df, X1_orig)
X3_orig_bd = join_and_drop_date(current_df, X3_orig_bd)

# Historia de curvas de forward tdc
x1_ftdc = join_and_drop_date(current_df, x1_ftdc, True)
x2_ftdc = join_and_drop_date(current_df, x2_ftdc, True)
X3_ftdc = join_and_drop_date(current_df, X3_ftdc)

# Historia de curvas de forward ind
X3_find = join_and_drop_date(current_df, X3_find)

# Historia de swaps
X1_orig_sw = join_and_drop_date(current_df, X1_orig_sw, True)
X2_orig_sw = join_and_drop_date(current_df, X2_orig_sw, True)

# Historia de opciones
x1_orig_oir = join_and_drop_date(current_df, x1_orig_oir, True)
x2_orig_oir = join_and_drop_date(current_df, x2_orig_oir, True)
x3_orig_oir = join_and_drop_date(current_df, x3_orig_oir)


Number of rows in lin_gub: 255
Number of rows in lin_gub_bmybdst: 255
Number of rows in lin_gub_bmybdst: 255
Number of rows in lin_gub_bmybdst_flib: 255
Number of rows in lin_gub_bmybdst_flibfwd: 255
Number of rows in lin_gub_bmybdst_flibfwdspind: 255
Number of rows in lin_gub_bmybdst_flibfwdspind_swcup: 255
Number of rows in lin_gub_bmybdst_flibfwdspind_swcupvp: 255
Number of rows in lin_gub_bmybdst_flibfwdspind_swcupvp_oirs: 255
Number of rows in lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvp: 255
Number of rows in lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol: 255
          Date
0   2022-09-28
1   2022-09-29
2   2022-09-30
3   2022-10-03
4   2022-10-04
..         ...
250 2023-09-25
251 2023-09-26
252 2023-09-27
253 2023-09-28
254 2023-09-29

[255 rows x 1 columns]


/tmp/ipython-input-2033783816.py:1: DeprecationWarning: numpy.core.fromnumeric is deprecated and has been renamed to numpy._core.fromnumeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.fromnumeric.sort.
  from numpy.core.fromnumeric import sort
/tmp/ipython-input-2033783816.py:2: DeprecationWarning: numpy.core.fromnumeric is deprecated and has been renamed to numpy._core.fromnumeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy AP

# Pasos para medir el riesgo por el método de Simulación Histórica

Para todos los instrumentos hay que definir y calcular los siguientes elementos:\
1.Historico de factores de riesgo\
2.Vector de precios actual\
3.Valoración al día actual

## Acciones y Divisas



In [8]:
m_fx=len(pos_fx)
m_acc=len(pos_eq)
M_acc_div=pos_eq+pos_fx
x0_acc_div=stock_prices_EQFX.iloc[0,:]
V0_acc_div=x0_acc_div*M_acc_div
print(x0_acc_div)
V0_acc_div

AMXB.MX            14.380024
GCARSOA1.MX       123.993317
WALMEX.MX          62.218273
BTC-USD        472003.066272
EURUSD=X           18.525420
USDMXN=X           17.538940
Name: 0, dtype: float64


,0
AMXB.MX,-71900.119781
GCARSOA1.MX,123993.316650
WALMEX.MX,74661.927795
BTC-USD,708004.599409
EURUSD=X,12967.793781
USDMXN=X,26308.410645


## Bondes D

In [9]:
m = len(plazos_bdm)
N_bd = (plazos_bdm // plazocupon_bdm) + 1  # número de cupones a pagar

# Inicialización de matrices/vectores
VTplazos_bdm = np.zeros(np.sum(N_bd))
contratos_bdmT = np.zeros(np.sum(N_bd))
plazocupon_bdmT = np.zeros(np.sum(N_bd))
tasafijaT_bd = np.zeros(np.sum(N_bd))
ulNomT_bd = np.zeros(np.sum(N_bd))
Xvp_bd = np.zeros((n, len(VTplazos_bdm)))
Xst_bd = np.zeros((n, len(VTplazos_bdm)))

# Cálculos adicionales
plazini_bd = plazos_bdm - plazocupon_bdm * (N_bd - 1)
ddv = plazocupon_bdm - plazini_bd
tfcupon = np.zeros(m)
tfcupondev = np.zeros(m)
tfcupgen = ((1 + tf_act / 360)**(plazocupon_bdm[0]) - 1) * 360 / plazocupon_bdm[0]

# Bucle para calcular tasas de cupón
for j in range(m):  # en Python los índices comienzan en 0
    tfcupondev[j] = ((np.prod(1 + tf_int[0:ddv[j]] / 360) - 1) * 360) / ddv[j]
    tfcupon[j] = (((1 + tfcupondev[j] * ddv[j] / 360) * (1 + tf_act / 360)**(plazocupon_bdm[0] - ddv[j]) - 1) * 360) / plazocupon_bdm[0]

print(tfcupondev)
print(tfcupon)

from scipy.interpolate import interp1d

for j in range(m):
    sum_N_bd = np.sum(N_bd[:j+1])
    sum_N_bd_prev = np.sum(N_bd[:j]) if j > 0 else 0

    if j == 0:
        VTplazos_bdm[:sum_N_bd] = np.arange(plazini_bd[j], plazos_bdm[j] + 1, plazocupon_bdm[j])
        contratos_bdmT[:sum_N_bd] = contratos_bdm[j]
        plazocupon_bdmT[:sum_N_bd] = plazocupon_bdm[j]
        ulNomT_bd[sum_N_bd - 1] = contratos_bdm[j]
        tasafijaT_bd[0] = tfcupon[j]
        tasafijaT_bd[1:sum_N_bd] = tfcupgen
    else:
        VTplazos_bdm[sum_N_bd_prev:sum_N_bd] = np.arange(plazini_bd[j], plazos_bdm[j] + 1, plazocupon_bdm[j])
        contratos_bdmT[sum_N_bd_prev:sum_N_bd] = contratos_bdm[j]
        plazocupon_bdmT[sum_N_bd_prev:sum_N_bd] = plazocupon_bdm[j]
        tasafijaT_bd[sum_N_bd_prev] = tfcupon[j]
        tasafijaT_bd[sum_N_bd_prev + 1:sum_N_bd] = tfcupgen
        ulNomT_bd[sum_N_bd - 1] = contratos_bdm[j]

for i in range(n):
    if itpl == 0:
        interpolator = interp1d(nodos, x_orig_gov.iloc[i, :], kind='linear', fill_value="interpolate")
        Xvp_bd[i, :] = interpolator(VTplazos_bdm)

        interpolator3 = interp1d(nodos3_bd, X3_orig_bd.iloc[i, :], kind='linear', fill_value="interpolate")
        Xst_bd[i, :] = interpolator3(VTplazos_bdm)
    else:
        # Código para interpolación 'talamb' (no definido en el ejemplo de R original)
        pass

X2_pr=pd.merge(X2_orig_bd, current_df, left_on='fecha', right_on="Date", how='inner').sort_values('Date', ascending=False)
X_bd_tc = (np.ones((n, len(contratos_bdmT))).T * (np.array(X2_pr[1]) / 100)).T
X_bd_ext = np.hstack((X_bd_tc, Xvp_bd, Xst_bd))


def bondeD(contratosT, nominal, tasafijaT, plazocuponT, VTplazos, Xvp, Xst, N, ulNomT):
    V0 = np.zeros(len(N))  # Inicializa V0, equivalente a matrix(0, 1, nrow(as.matrix(N))) en R
    V0f = (((contratosT * tasafijaT * (plazocuponT / 360)) + ulNomT) / (1 + (Xvp + Xst) * VTplazos / 360)) * nominal

    for j in range(len(N)):  # Cambia índices de 1:nrow(as.matrix(N)) a 0:(len(N)-1)
        if j == 0:  # Cambia de j==1 a j==0
            V0[j] = np.sum(V0f[:N[j]])
        else:
            V0[j] = np.sum(V0f[np.sum(N[:j]):np.sum(N[:j+1])])

    return V0

# Suponiendo que contratosT, nominal, tasafijaT, plazocuponT, VTplazos, Xvp, Xst, y N ya están definidos
V0_bd = bondeD(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[0,], Xst_bd[0,], N_bd, ulNomT_bd)  # Cambia de Xvp[1, ] y Xst[1, ] a Xvp[0, :] y Xst[0, :]

print(V0_bd)

[0.11001729 0.11038289 0.11022157 0.10992531 0.11044562]
[0.10980679 0.11015967 0.10996162 0.10974557 0.11031451]
[ 22161.21115235 -29320.69510623  29377.07811909 -47181.61323033
  10480.40284521]


## Forwards de Tipo de Cambio

In [10]:
import numpy as np

# Suponemos que plazos_fwd, n, itpl, nodos1_ftdc, nodos2_ftdc, x1_ftdc, x2_ftdc, trlib,
# X3_ftdc, kst_fwd, contratos_fwd, y nominal_fwd ya están definidos

m = len(plazos_fwd)

X1_fwtdc = np.zeros((n, m))
X2_fwtdc = np.zeros((n, m))

def approx(x, y, new_x, rule=2):
    return np.interp(new_x, x, y)


for j in range(n):
    if itpl == 0:
        X1_fwtdc[j, :] = approx(nodos1_ftdc, x1_ftdc.iloc[j, :], plazos_fwd)
        X2_fwtdc[j, :] = approx(nodos2_ftdc, x2_ftdc.iloc[j, :], plazos_fwd)
    else:
        X1_fwtdc[j, :] = talamb(nodos1_ftdc, x1_ftdc.iloc[j, :], plazos_fwd)
        X2_fwtdc[j, :] = talamb(nodos2_ftdc, x2_ftdc.iloc[j, :], plazos_fwd)

    if trlib == 1:
        X1_fwtdc[j, :] = ((1 + X1_fwtdc[j, :]) ** (plazos_fwd / 180) - 1) * 360 / plazos_fwd

def futuroTC(t, tl, tn, s, k):
    f = s * ((1 + tn * t / 360) / (1 + tl * t / 360))
    return (f - k) / (1 + t * tn / 360)

X3_ftdc = np.array(X3_ftdc)  # Aseguramos que sea una matriz de NumPy
X_futtdc = np.hstack((X1_fwtdc, X2_fwtdc, X3_ftdc))

V0_fwtdc = futuroTC(plazos_fwd, X1_fwtdc[0, :], X2_fwtdc[0, :], X3_ftdc[0, :], kst_fwd) * contratos_fwd * nominal_fwd
print(V0_fwtdc)

[-1110.46395701   533.68768807  -862.82287288  1036.76067149
 -1121.6482204 ]


## FORWARD DE INDICES

In [11]:
m_ind = len(plazos_fwd_ind)

X1_fwind = np.zeros((n, m_ind))  # Matriz para DIVIDENDOS
X2_fwind = np.zeros((n, m_ind))

def approx(x, y, new_x, rule=2):
    return np.interp(new_x, x, y)


for j in range(n):
    if itpl == 0:
        X2_fwind[j, :] = approx(nodos_gov, x_orig_gov.iloc[j, :], plazos_fwd_ind)
    else:
        X2_fwind[j, :] = talamb(nodos_gov, x_orig_gov.iloc[j, :], plazos_fwd_ind)
X3_find = np.array(X3_find)

X_futind = np.hstack((X1_fwind, X2_fwind, X3_find))


V0_fwind = futuroTC(plazos_fwd_ind, X1_fwind[0, :], X2_fwind[0, :], X3_find[0, :], kst_fwd_ind) * contratos_fwd_ind * nominal_fwd_ind


In [12]:
V0_fwind

array([ 13916.8821241 ,  -2944.15147144, 109426.77479725])

## SWAPS

In [13]:
nodosvp=nodos1_sw
nodostc=nodos2_sw
curvavp=X1_orig_sw
curvatc=X2_orig_sw
n1=len(curvavp)
n2=len(curvatc)


m = len(plazos_sw)
N = (plazos_sw // plazocupon_sw) + 1  # número de cupones a pagar

# Inicialización de matrices/vectores
VTplazos_sw = np.zeros(np.sum(N))
VTplazos_swc = np.zeros(np.sum(N))
contratos_swT = np.zeros(np.sum(N))
por_swT=np.zeros(sum(N)) #vector de todos los dummy si paga o recibe de todos los flujos de todos los contratos_sw
plazocupon_swT = np.zeros(np.sum(N))
tasafija_swT = np.zeros(np.sum(N))
nominal_swT = np.zeros(np.sum(N))
Xvp = np.zeros((n, len(VTplazos_sw)))
Xtc = np.zeros((n, len(VTplazos_sw)))
Xtcc = np.zeros((n, len(VTplazos_sw)))
XtfwdT = np.zeros((n, len(VTplazos_sw)))

# Cálculos adicionales
plazini_sw = plazos_sw - plazocupon_sw * (N - 1)

from scipy.interpolate import interp1d

for j in range(m):
    sum_N = np.sum(N[:j+1])
    sum_N_prev = np.sum(N[:j]) if j > 0 else 0

    if j == 0:
        VTplazos_sw[:sum_N] = np.arange(plazini_sw[j], plazos_sw[j] + 1, plazocupon_sw[j])
        VTplazos_swc[:sum_N] = np.concatenate(([0], VTplazos_sw[0:(sum_N - 1)]))
        contratos_swT[:sum_N] = contratos_sw[j]
        plazocupon_swT[:sum_N] = plazocupon_sw[j]
        nominal_swT[:sum_N] = nominal_sw[j]
        por_swT[:sum_N] = por_sw[j]
        tasafija_swT[:sum_N] = tasafija_sw[j]
    else:
        VTplazos_sw[sum_N_prev:sum_N] = np.arange(plazini_sw[j], plazos_sw[j] + 1, plazocupon_sw[j])
        VTplazos_swc[sum_N_prev:sum_N] = np.concatenate(([0], VTplazos_sw[sum_N_prev:(sum_N - 1)]))
        contratos_swT[sum_N_prev:sum_N] = contratos_sw[j]
        plazocupon_swT[sum_N_prev:sum_N] = plazocupon_sw[j]
        nominal_swT[sum_N_prev:sum_N] = nominal_sw[j]
        por_swT[sum_N_prev:sum_N] = por_sw[j]
        tasafija_swT[sum_N_prev:sum_N] = tasafija_sw[j]


for i in range(n):
    if itpl == 0:
        Xvp[i, :] = approx(nodosvp, curvavp.iloc[i, :], VTplazos_sw)
        Xtc[i, :] = approx(nodostc, curvatc.iloc[i, :], VTplazos_sw)
        Xtcc[i, :] = approx(nodostc, curvatc.iloc[i, :], VTplazos_swc)
    else:
        Xvp[i, :] = talamb(nodosvp, curvavp.iloc[i, :], VTplazos_sw)
        Xtc[i, :] = talamb(nodostc, curvatc.iloc[i, :], VTplazos_sw)
        Xtcc[i, :] = talamb(nodostc, curvatc.iloc[i, :], VTplazos_swc)

    XtfwdT[i, :] = ((1 + Xtc[i, :] * VTplazos_sw / 360) / (1 + Xtcc[i, :] * VTplazos_swc / 360) - 1) * 360 / plazocupon_swT

    j = 0
    while j < len(VTplazos_sw):
        if VTplazos_sw[j] <= plazocupon_swT[j]:
            XtfwdT[i, j] = Xtc[i, j]
        else:
            j = np.sum(N[0:(j+1)]) - 1
        j += 1

X_sw = np.hstack((XtfwdT, Xvp))

def swap(por_swT, contratos_swT, nominal_swT, XtfwdT, tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp, N):
    V0 = np.zeros(len(N))
    V0f = (contratos_swT * (XtfwdT - tasafija_swT) * (plazocupon_swT / 360)) / ((1 + Xvp * VTplazos_sw / 360) * nominal_swT * ((-1) ** por_swT))

    for j in range(len(N)):
        if j == 0:
            V0[j] = np.sum(V0f[0:N[j]])
        else:
            V0[j] = np.sum(V0f[np.sum(N[0:j]):np.sum(N[0:(j+1)])])

    return V0

V0_sw = swap(por_swT, contratos_swT, nominal_swT, XtfwdT[0, :], tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp[0, :], N)
print(V0_sw)


[3.40230964 4.10791942]


##Opciones de Tasa de Interés

In [14]:
#from numpy.core.multiarray import array
#x1[i, :] = np.interp(plazos_oir, nodos1_oir.index, x1_orig_oir.iloc[i, :-1].astype("float64")/100)
# print( x1_orig_oir.iloc[i, :-1].astype("float64"))
print(nodos1_oir.index)

import numpy as np
import scipy.stats as stats

# Opciones de tasa de interés, cálculo con inicio el día de la valuación

# Posición inicial

# Interpolación de tasas y volatilidades
m = len(plazos_oir)
x1 = np.zeros((n, m))
x2tc = np.zeros((n, m))
x2tl = np.zeros((n, m))
x2 = np.zeros((n, m))
x3 = np.zeros((n, m))
for i in range(n-1):
    x1[i, :] = np.interp(plazos_oir, nodos1_oir, x1_orig_oir.iloc[i, :])
    x2tc[i, :] = np.interp(plazos_oir, nodos2_oir, x2_orig_oir.iloc[i, :])
    x2tl[i, :] = np.interp(plazos_oir + pr_oir, nodos2_oir, x2_orig_oir.iloc[i, :])
    x3[i, :] = np.interp(plazos_oir, nodos3_oir, x3_orig_oir.iloc[i, :])
    x2[i, :] = (((1 + x2tl[i, :] * ((plazos_oir + pr_oir) / 360)) / (1 + x2tc[i, :] * (plazos_oir / 360))) - 1) * 360 / pr_oir

x01 = x1[0, :]  # Tasas de descuento
x02 = x2[0, :]  # Tasas spot
x03 = x3[0, :]  # Volatilidades


X_oir = np.column_stack((x1, x2, x3))
def opctint(d, S, K_oir, vol, t, cp_oir, cs_oir, pr_oir, dct_oir):
    d1 = ((np.log(S / K_oir) + (vol ** 2) * t / (365 * 2)) * (1 / (vol * np.sqrt(t / 365)))) if cs_oir == 1 else ((np.log(S / K_oir) + (vol ** 2) * t / (360 * 2)) * (1 / (vol * np.sqrt(t / 360))))
    d2 = ((np.log(S / K_oir) - (vol ** 2) * t / (365 * 2)) * (1 / (vol * np.sqrt(t / 365)))) if cs_oir == 1 else ((np.log(S / K_oir) - (vol ** 2) * t / (360 * 2)) * (1 / (vol * np.sqrt(t / 360))))
    vp = (np.log(1 + d * t / 360) * 365 / t) if cs_oir == 1 else d
    f=((S * stats.norm.cdf(d1 * (-1) ** cp_oir) - K_oir * stats.norm.cdf(d2 * (-1) ** cp_oir)) * (np.exp(-vp * t / 365)) * (-1) ** cp_oir)* (pr_oir / dct_oir) / (1 + S * pr_oir / dct_oir) if cs_oir == 1 else ((S * stats.norm.cdf(d1 * (-1) ** cp_oir) - K_oir * stats.norm.cdf(d2 * (-1) ** cp_oir)) * (1 / (1 + vp * t / 360)) * (-1) ** cp_oir) * (pr_oir / dct_oir) / (1 + S * pr_oir / dct_oir)
    return f

V0_oir = opctint(x01, x02, K_oir, x03, plazos_oir, cp_oir, cs_oir, pr_oir, dct_oir) * contratos_oir * nominal_oir  # Valor del portafolio
V0_oir


Index([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], dtype='int64')


array([ 1.52929996, -1.01894708,  1.05851526, -0.10334304,  2.31481121])

# Integración de factores y cálculo de riesgo en conjunto, y aplicación de simulación

In [15]:
import numpy as np

# Suponemos que todas las matrices como stock_prices_EQFX, X_bd_ext, etc., ya están definidas

# Número de columnas para cada conjunto de datos
n_if = np.zeros((6, 1))
n_if[0] = stock_prices_EQFX.shape[1]  # acciones y divisas
n_if[1] = X_bd_ext.shape[1]  # bonde
n_if[2] = X_sw.shape[1]  # swaps
n_if[3] = X_oir.shape[1]  # opciones tasa de interés
n_if[4] = X_futtdc.shape[1]  # Forwards de tipo de cambio
n_if[5] = X_futind.shape[1]  # Forwards de índices

n_if = n_if.astype(int)

# Valor del portafolios
V0_port = np.hstack((V0_acc_div, V0_bd, V0_sw, V0_oir, V0_fwtdc, V0_fwind))
V0T_port = np.sum(V0_port)

# Integración de todos los factores de riesgo en una matriz
X_port = np.hstack((stock_prices_EQFX, X_bd_ext, X_sw, X_oir, X_futtdc, X_futind))

# Cálculo de variaciones Delta_X del portafolios
DeltaX_port = np.log(X_port[0:(n - 1), :] / X_port[1:n, :])
DeltaX_port[np.isnan(DeltaX_port)] = 0  # quitamos NaN
DeltaX_port[np.isinf(DeltaX_port)] = 0  # quitamos Infinitos

# Número de escenarios históricos
Ns = DeltaX_port.shape[0]

# A continuación, las impresiones en R se traducen a Python
print(DeltaX_port.shape[1])  # ncol(DeltaX_s)
print(DeltaX_port.shape[0])  # nrow(DeltaX_s)
print(n_if)
print(DeltaX_port.shape)

577
254
[[  6]
 [468]
 [ 70]
 [ 15]
 [ 11]
 [  7]]
(254, 577)


/tmp/ipython-input-3496566507.py:24: RuntimeWarning: divide by zero encountered in divide
  DeltaX_port = np.log(X_port[0:(n - 1), :] / X_port[1:n, :])
/tmp/ipython-input-3496566507.py:24: RuntimeWarning: invalid value encountered in divide
  DeltaX_port = np.log(X_port[0:(n - 1), :] / X_port[1:n, :])


# Medición de Riesgo

## Acciones y divisas

In [24]:
import numpy as np
import pandas as pd

# Inicialización de variables
m = m_fx + m_acc
# Ns = DeltaX_port.shape[0]
n_if_1 = n_if[0]

# Inicializar matrices con ceros
X_s_acc_div = np.zeros((Ns, n_if_1[0])) # Para simular los precios X*
V_acc_div = np.zeros((Ns, m)) # Valor de Y con X*
Vfr1_acc_div = np.zeros((Ns, 1)) # Valor de Y* solo de tipo de cambio o divisas
Vfr2_acc_div = np.zeros((Ns, 1)) # Valor de Y solo de tipo de acciones
PG_acc_div = np.zeros((Ns, m)) # Matriz de \Delta Y = Y - Y0 Pérdidas y Ganancias
PGfr1_acc_div = np.zeros((Ns, 1))  # Matriz de \Delta Y = Y - Y0 Pérdidas y Ganancias de divisas
PGfr2_acc_div = np.zeros((Ns, 1)) # Matriz de \Delta Y = Y - Y0 Pérdidas y Ganancias de acciones
PGT_acc_div = np.zeros(Ns) # Matriz de \Delta Y = Y - Y0 Pérdidas y Ganancias totales

DeltaX_s_acc_div = DeltaX_port[:, :n_if_1[0]] # Matriz de \Delta X de factores de riesgoo acciones y divisas

x0_acc_div = stock_prices_EQFX.loc[0]

# Loop principal
for i in range(Ns):
    X_s_acc_div[i, :] = x0_acc_div * np.exp(DeltaX_s_acc_div[i]) # X*
    V_acc_div[i, :] = np.hstack((pos_eq, pos_fx)) * X_s_acc_div[i, :] # Y
    Vfr1_acc_div[i, :] = sum(V_acc_div[i, :m_acc]) # Y divisas
    Vfr2_acc_div[i, :] = sum(V_acc_div[i, m_acc:]) # Y acciones
    PG_acc_div[i, :] = V_acc_div[i, :] - V0_acc_div # \Delta Y
    PGfr1_acc_div[i, :] = Vfr1_acc_div[i] - sum(V0_acc_div[:m_acc])  # \Delta Y DE DIVISAS
    PGfr2_acc_div[i, :] = Vfr2_acc_div[i] - sum(V0_acc_div[m_acc:]) # \Delta Y de acciones
    PGT_acc_div[i] = np.sum(PG_acc_div[i, :]) # \Delta Y Totales

# Calculo de VaR y CVaR
VaRCont_acc_div = np.quantile(PG_acc_div, 1-alpha, axis=0)
#CVaRCont_acc_div = np.mean(PG_acc_div[PG_acc_div < VaRCont_acc_div], axis=0)

CVaR_temp = []
for i in range(len(PG_acc_div[0])):
  valor = []
  for j in range(0,len(PG_acc_div[PG_acc_div[:,i] < VaRCont_acc_div[i]])):
    valor += [PG_acc_div[PG_acc_div[:,i] < VaRCont_acc_div[i]][j][i]]
  CVaR_temp += [np.mean(valor)]
CVaRCont_acc_div = np.array(CVaR_temp)

VaRPort_acc_div = np.quantile(PGT_acc_div, 1-alpha, axis=0)
CVaRPort_acc_div = np.mean(PGT_acc_div[PGT_acc_div < VaRPort_acc_div], axis=0)

VaRCont_fr1 = np.quantile(sum(PGfr1_acc_div), 1-alpha, axis=0)
VaRCont_fr2 = np.quantile(sum(PGfr2_acc_div), 1-alpha, axis=0)

# Hagamos lo mismo con la función equantile
VaR_Cont_acc_div_e = [equantile(PG_acc_div[:,i], 1-alpha) for i in range(PG_acc_div.shape[1])]
CVaRCont_acc_div_e = [np.mean(PG_acc_div[PG_acc_div[:,i]<VaR_Cont_acc_div_e[i]]) for i in range(PG_acc_div.shape[1])]

# Generaré las mismas medidas pero para con alisado
# ayúdame a generarlas
VaR_Cont_acc_div_ca = [wquantile(PG_acc_div[:,i],w0,1-alpha) for i in range(PG_acc_div.shape[1])]
CVaRCont_acc_div_ca =  [wcvar(PG_acc_div[:,i],w0,1-alpha) for i in range(PG_acc_div.shape[1])]

In [17]:
CVaRCont_acc_div_e

[np.float64(-1165.5144689675135),
 np.float64(-2893.7285151496703),
 np.float64(-59.08906041196042),
 np.float64(-14748.190787091267),
 np.float64(-1557.7549597163295),
 np.float64(-2290.6643612981466)]

In [20]:
 wquantile(PG_acc_div[:,0],w0,1-alpha)

np.float64(-1304.9928702106342)

In [ ]:
print(pd.DataFrame(V_acc_div).head(2))

pd.DataFrame(Vfr1_acc_div).head(2)

              0              1             2              3             4  \
0 -74951.194867  123953.547051  78541.973065  699226.254935  12929.225159   
1 -76567.505681  125896.428727  79055.578586  731846.913457  12993.327686   

              5  
0  26088.253162  
1  26521.248600  


,0
0,127544.325250
1,128384.501632


In [ ]:
# pd.DataFrame(X_s_acc_div[0:10,:])
print("VaR por posición")
print(VaRCont_acc_div)
print("CVaR por posición")
print(CVaRCont_acc_div)
print("VaR Total de acciones y divisas")
print(VaRPort_acc_div)
print("CVaR Total de acciones y divisas")
print(CVaRPort_acc_div)

print("VaR Total de acciones")
print(VaRCont_fr1)
print("VaR Total de divisas")
print(VaRCont_fr2)
# # print(np.hstack((pos_fx, pos_eq)))

# # pd.DataFrame(PG_acc_div)

# print(V0_acc_div)

VaR por posición
[ -1967.45866244  -4624.28340645  -2286.32461372 -38158.6987366
   -178.4163708    -360.30537065]
CVaR por posición
[ -3098.81845212  -6012.64106745  -2570.35982465 -59881.75896391
   -198.74043417   -392.87873726]
VaR Total de acciones y divisas
-42985.95866077218
CVaR Total de acciones y divisas
-61982.079828087684
VaR Total de acciones
76349.46802694732
VaR Total de divisas
203501.2794957012


In [ ]:
np.quantile(PGT_acc_div, 1-alpha, axis=0)

-2521255952.785253

In [ ]:
np.mean(PGT_acc_div[PGT_acc_div < VaRPort_acc_div], axis=0)

-2738287291.1819067

In [ ]:
-4.11997670e+04

-41199.767

In [ ]:
-4.21678484e+04

-42167.8484

## Bonde

In [ ]:
# Translating the given R code to Python
import numpy as np

# Assuming Ns, n_if, x0_bd, contratos_bdmT, nominal_bdm, plazocupon_bdmT, VTplazos_bdm, N_bd, and ddv are defined

m = len(N_bd)  # Key step
X_s_bd = np.zeros((Ns, int(n_if[1])))  # Simulated risk factors based on DeltaX_s x0*(1+Delta_Xs)  # Key step
V_bd = np.zeros((Ns, m))
Vfr1_bd = np.zeros((Ns, m))
Vfr2_bd = np.zeros((Ns, m))
Vfr3_bd = np.zeros((Ns, m))
PG_bd = np.zeros((Ns, m))  # Losses and gains
PGfr1_bd = np.zeros((Ns, m))
PGfr2_bd = np.zeros((Ns, m))
PGfr3_bd = np.zeros((Ns, m))
PGT_bd = np.zeros((Ns, 1))
PGfr1T_bd = np.zeros((Ns, 1))
PGfr2T_bd = np.zeros((Ns, 1))
PGfr3T_bd = np.zeros((Ns, 1))

# Assuming DeltaX_s is defined
DeltaX_s_bd = DeltaX_port[:, int(sum(n_if[:1])) : int(sum(n_if[:2]))]  # Key step

# Assuming x0_bd is defined
x0_bd = X_bd_ext[0, :]  # Key step

for i in range(Ns):
    X_s_bd[i, :] = x0_bd * np.exp(DeltaX_s_bd[i, :])
    # Key step
    # Assuming bondeD is a defined function
    V_bd[i, :] = bondeD(
        contratos_bdmT,
        nominal_bdm,
        X_s_bd[i, 0],
        plazocupon_bdmT,
        VTplazos_bdm,
        X_s_bd[i, int(n_if[1] / 3) : int(n_if[1] * 2 / 3)],
        X_s_bd[i, int(n_if[1] * 2 / 3) : int(n_if[1])],
        N_bd,
        ulNomT_bd,
    )
    # Key step
    Vfr1_bd[i, :] = bondeD(
        contratos_bdmT,
        nominal_bdm,
        X_s_bd[i, 0],
        plazocupon_bdmT,
        VTplazos_bdm,
        x0_bd[int(n_if[1] / 3) : int(n_if[1] * 2 / 3)],
        x0_bd[int(n_if[1] * 2 / 3) : int(n_if[1])],
        N_bd,
        ulNomT_bd,
    )
    # Key step
    Vfr2_bd[i, :] = bondeD(
        contratos_bdmT,
        nominal_bdm,
        x0_bd[0],
        plazocupon_bdmT,
        VTplazos_bdm,
        X_s_bd[i, int(n_if[1] / 3) : int(n_if[1] * 2 / 3)],
        x0_bd[int(n_if[1] * 2 / 3) : int(n_if[1])],
        N_bd,
        ulNomT_bd,
    )
    # Key step
    Vfr3_bd[i, :] = bondeD(
        contratos_bdmT,
        nominal_bdm,
        x0_bd[0],
        plazocupon_bdmT,
        VTplazos_bdm,
        x0_bd[int(n_if[1] / 3) : int(n_if[1] * 2 / 3)],
        X_s_bd[i, int(n_if[1] * 2 / 3) : int(n_if[1])],
        N_bd,
        ulNomT_bd,
    )
    # Key step
    PG_bd[i, :] = V_bd[i, :] - V0_bd
    PGfr1_bd[i, :] = Vfr1_bd[i, :] - V0_bd
    PGfr2_bd[i, :] = Vfr2_bd[i, :] - V0_bd
    PGfr3_bd[i, :] = Vfr3_bd[i, :] - V0_bd
    PGT_bd[i, 0] = np.sum(PG_bd[i, :])
    PGfr1T_bd[i, 0] = np.sum(PGfr1_bd[i, :])
    PGfr2T_bd[i, 0] = np.sum(PGfr2_bd[i, :])
    PGfr3T_bd[i, 0] = np.sum(PGfr3_bd[i, :])



VaRCont_bd = np.zeros(m)
VaRfr1_bd = np.zeros(m)
VaRfr2_bd = np.zeros(m)
VaRfr3_bd = np.zeros(m)
CVaRfr1_bd = np.zeros(m)
CVaRfr2_bd = np.zeros(m)
CVaRfr3_bd = np.zeros(m)
CVaRCont_bd = np.zeros(m)

# Ciclo for para calcular VaR y CVaR
for i in range(m):
    VaRCont_bd[i] = np.percentile(PG_bd[:, i], (1 - alpha), axis=0)
    VaRfr1_bd[i] = np.percentile(PGfr1_bd[:, i], (1 - alpha), axis=0)
    VaRfr2_bd[i] = np.percentile(PGfr2_bd[:, i], (1 - alpha), axis=0)
    VaRfr3_bd[i] = np.percentile(PGfr3_bd[:, i], (1 - alpha), axis=0)

    CVaRfr1_bd[i] = np.mean(PGfr1_bd[PGfr1_bd[:, i] < VaRfr1_bd[i], i])
    CVaRfr2_bd[i] = np.mean(PGfr2_bd[PGfr2_bd[:, i] < VaRfr2_bd[i], i])
    CVaRfr3_bd[i] = np.mean(PGfr3_bd[PGfr3_bd[:, i] < VaRfr3_bd[i], i])
    CVaRCont_bd[i] = np.mean(PG_bd[PG_bd[:, i] < VaRCont_bd[i], i])

# Imprimir resultados
print("VaRCont_bd:", VaRCont_bd)
print("VaRfr1_bd:", VaRfr1_bd)
print("VaRfr2_bd:", VaRfr2_bd)
print("VaRfr3_bd:", VaRfr3_bd)
print("CVaRCont_bd:", CVaRCont_bd)
print("CVaRfr1_bd:", CVaRfr1_bd)
print("CVaRfr2_bd:", CVaRfr2_bd)
print("CVaRfr3_bd:", CVaRfr3_bd)

# VaR Total
VaRTotal_bd = np.percentile(PGT_bd, (1 - alpha) * 100)
VaRTotalfr1_bd = np.percentile(np.sum(PGfr1_bd, axis=1), (1 - alpha) * 100)
VaRTotalfr2_bd = np.percentile(np.sum(PGfr2_bd, axis=1), (1 - alpha) * 100)
VaRTotalfr3_bd = np.percentile(np.sum(PGfr3_bd, axis=1), (1 - alpha) * 100)

# CVaR Total
CVaRTotal_bd = np.mean(PGT_bd[PGT_bd < VaRTotal_bd])
CVaRTotalfr1_bd = np.mean(np.sum(PGfr1_bd, axis=1)[np.sum(PGfr1_bd, axis=1) < VaRTotalfr1_bd])
CVaRTotalfr2_bd = np.mean(np.sum(PGfr2_bd, axis=1)[np.sum(PGfr2_bd, axis=1) < VaRTotalfr2_bd])
CVaRTotalfr3_bd = np.mean(np.sum(PGfr3_bd, axis=1)[np.sum(PGfr3_bd, axis=1) < VaRTotalfr3_bd])

print("VaRTotal_bd:", VaRTotal_bd)
print("VaRTotalfr1_bd:", VaRTotalfr1_bd)
print("VaRTotalfr2_bd:", VaRTotalfr2_bd)
print("VaRTotalfr3_bd:", VaRTotalfr3_bd)
print("CVaRTotalCont_bd:", CVaRTotal_bd)
print("CVaRTotalfr1_bd:", CVaRTotalfr1_bd)
print("CVaRTotalfr2_bd:", CVaRTotalfr2_bd)
print("CVaRTotalfr3_bd:", CVaRTotalfr3_bd)
print(V0_bd)
# Imprimir resultados
# print(np.c_[VaRTotal_bd, np.sum(V0_bd), VaRCont_bd, V0_bd])
# print(np.c_[CVaRTotal_bd, np.sum(V0_bd), CVaRCont_bd, V0_bd])
# print(np.c_[VaRTotal_bd, VaRTotalfr1_bd, VaRTotalfr2_bd, VaRTotalfr3_bd])
# print(np.c_[CVaRTotal_bd, CVaRTotalfr1_bd, CVaRTotalfr2_bd, CVaRTotalfr3_bd])

<ipython-input-15-f131f79aed7f>:7: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  X_s_bd = np.zeros((Ns, int(n_if[1])))  # Simulated risk factors based on DeltaX_s x0*(1+Delta_Xs)  # Key step
<ipython-input-15-f131f79aed7f>:22: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  DeltaX_s_bd = DeltaX_port[:, int(sum(n_if[:1])) : int(sum(n_if[:2]))]  # Key step
<ipython-input-15-f131f79aed7f>:37: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  X_s_bd[i, int(n_if[1] / 3) : int

VaRCont_bd: [ 5.06745805e-01 -4.54323911e+02  5.61664787e+01 -1.84725655e+03
 -2.91195069e+01]
VaRfr1_bd: [   46.83876912  -445.94683889    92.56079406 -1820.76939363
    88.36581457]
VaRfr2_bd: [  11.84931015 -209.34557416   72.69323003 -843.43179669   24.05916773]
VaRfr3_bd: [  53.08020203 -144.21945297  120.29651435 -598.32142894   96.51385707]
CVaRCont_bd: [-8.37093144e-01 -4.55645172e+02  5.57649648e+01 -1.85037654e+03
 -3.19222269e+01]
CVaRfr1_bd: [   46.74813896  -447.46131394    92.38169456 -1826.95288812
    88.1948321 ]
CVaRfr2_bd: [  10.67644545 -211.55755289   71.37073187 -844.25232745   22.05265291]
CVaRfr3_bd: [  52.68844626 -144.96194791  119.85939781 -598.61146048   95.76323498]
VaRTotal_bd: -455.6889666184792
VaRTotalfr1_bd: -301.4483820927352
VaRTotalfr2_bd: -317.27346353637466
VaRTotalfr3_bd: -249.39195413958814
CVaRTotalCont_bd: -609.0737249232867
CVaRTotalfr1_bd: -560.785714638414
CVaRTotalfr2_bd: -363.18253215660087
CVaRTotalfr3_bd: -266.55315518751894
[ 22161.211

## Swaps

In [ ]:
# Translating the given R code to Python
import numpy as np

# Assuming Ns, n_if, x0_bd, contratos_bdmT, nominal_bdm, plazocupon_bdmT, VTplazos_bdm, N_bd, and ddv are defined
m = N.shape[0]  # Key step


X_s_sw = np.zeros((Ns, int(n_if[2])))  # Simulated risk factors based on DeltaX_s x0*(1+Delta_Xs)  # Key step
V_sw = np.zeros((Ns, m))
Vfr1_sw = np.zeros((Ns,m))
Vfr2_sw = np.zeros((Ns, m))
PG_sw = np.zeros((Ns, m))  # Losses and gains
PGfr1_sw = np.zeros((Ns, m))
PGfr2_sw = np.zeros((Ns, m))
PGT_sw = np.zeros((Ns, m))
PGfr1T_sw = np.zeros((Ns, m))
PGfr2T_sw = np.zeros((Ns, m))

#N_sw
DeltaX_s_sw = DeltaX_port[:, int(sum(n_if[:2])) : int(sum(n_if[:3]))]  # Key step
x0_sw = X_sw[0, :]  # Key step

for i in range(Ns):
  X_s_sw[i,:] = x0_sw * np.exp(DeltaX_s_sw[i])
  # PASO CLAVE
  V_sw[i] = swap(por_swT, contratos_swT, nominal_swT, X_s_sw[i,:(n_if[2][0]//2)], tasafija_swT, plazocupon_swT, VTplazos_sw, X_s_sw[i,(n_if[2][0]//2):n_if[2][0]], N)
  # PASO CLAVE
  Vfr1_sw[i] = swap(por_swT, contratos_swT, nominal_swT, X_s_sw[i,:(n_if[2][0]//2)], tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp[0,:], N)
  # PASO CLAVE
  Vfr2_sw[i] = swap(por_swT, contratos_swT, nominal_swT,  XtfwdT[i,:], tasafija_swT, plazocupon_swT, VTplazos_sw, X_s_sw[i,(n_if[2][0]//2):n_if[2][0]], N)
  #PASO CLAVE
  PG_sw[i] = V_sw[i] - V0_sw
  PGfr1_sw[i] = Vfr1_sw[i] - V0_sw
  PGfr2_sw[i] = Vfr2_sw[i] - V0_sw
  PGT_sw[i] = np.sum(PG_sw)
  PGfr1T_sw[i] = np.sum(PGfr1_sw[i])
  PGfr2T_sw[i] = np.sum(PGfr2_sw[i])

# VaR por posición
VaRCont_sw = np.zeros((1, m))[0]
VaRfr1_sw = np.zeros((1, m))[0]
VaRfr2_sw = np.zeros((1, m))[0]
CVaRCont_sw = np.zeros((1, m))[0]
CVaRfr1_sw = np.zeros((1, m))[0]
CVaRfr2_sw = np.zeros((1, m))[0]

for i in range(m):
  VaRCont_sw[i] = np.quantile(PG_sw[:, i], 1-alpha)
  VaRfr1_sw[i] = np.quantile(PGfr1_sw[:, i], 1-alpha)
  VaRfr2_sw[i] = np.quantile(PGfr2_sw[:, i], 1-alpha)

  CVaRfr1_sw[i] = np.mean(np.column_stack((np.arange(0, Ns),PGfr1_sw[:, i]))[np.where(PGfr1_sw[:, i] < VaRfr1_sw[i])[0],1])
  CVaRfr2_sw[i] = np.mean(np.column_stack((np.arange(0, Ns),PGfr2_sw[:, i]))[np.where(PGfr2_sw[:, i] < VaRfr2_sw[i])[0],1])
  CVaRCont_sw[i] = np.mean(np.column_stack((np.arange(0, Ns),PG_sw[:, i]))[np.where(PG_sw[:, i] < VaRCont_sw[i])[0],1])

VaRCont_sw
VaRfr1_sw
VaRfr2_sw
CVaRCont_sw
CVaRfr1_sw
CVaRfr2_sw

VaRTotal_sw = np.quantile(PGT_sw, 1-alpha)
VaRTotalfr1_sw = np.quantile(PGfr1T_sw, 1-alpha)
VaRTotalfr2_sw= np.quantile(PGfr2T_sw, 1-alpha)

CVaRTotal_sw= np.mean(np.column_stack((np.arange(0, Ns),PGT_sw))[np.where(PGT_sw < VaRTotal_sw)[0], np.where(PGT_sw < VaRTotal_sw)[1]][1:][::2])
CVaRTotalfr1_sw= np.mean(PGfr1T_sw[np.where(PGfr1T_sw < VaRTotalfr1_sw)])
CVaRTotalfr2_sw= np.mean(PGfr2T_sw[np.where(PGfr2T_sw < VaRTotalfr2_sw)])

print("VaRCont_sw:", VaRCont_sw)
print("VaRfr1_sw:", VaRfr1_sw)
print("VaRfr2_sw:", VaRfr2_sw)
print("CVaRCont_sw:", CVaRCont_sw)
print("CVaRfr1_sw:", CVaRfr1_sw)
print("CVaRfr2_sw:", CVaRfr2_sw)

print("VaR Total:", VaRTotal_sw)
print("VaR Total fr1:", VaRTotalfr1_sw)
print("VaR Total fr2:", VaRTotalfr2_sw)
print("CVaR Total:", CVaRTotal_sw)
print("CVaR Total fr1:", CVaRTotalfr1_sw)
print("CVaR Total fr2:", CVaRTotalfr2_sw)
#Los de instrumentos estan en el último for

VaRCont_sw: [-1.76781469 -0.97908154]
VaRfr1_sw: [-1.76647183 -0.98026004]
VaRfr2_sw: [-4.76626838 -3.87296662]
CVaRCont_sw: [-3.01416    -1.33172174]
CVaRfr1_sw: [-3.01286391 -1.33180521]
CVaRfr2_sw: [-5.45792973 -4.02325129]
VaR Total: -7.854654481017954
VaR Total fr1: -1.438148654175611
VaR Total fr2: -4.327890598728387
CVaR Total: -8.066712567847807
CVaR Total fr1: -2.3872487515119447
CVaR Total fr2: -5.077816095515877


<ipython-input-17-228554a06161>:8: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  X_s_sw = np.zeros((Ns, int(n_if[2])))  # Simulated risk factors based on DeltaX_s x0*(1+Delta_Xs)  # Key step
<ipython-input-17-228554a06161>:20: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  DeltaX_s_sw = DeltaX_port[:, int(sum(n_if[:2])) : int(sum(n_if[:3]))]  # Key step


## Opciones de tasa de interés

In [ ]:
m = len(plazos_oir)  # Número de columnas en plazos_oir (PASO CLAVE)
X_s_oir = np.zeros((Ns, n_if[3][0]))  # Factores de riesgo simulados (PASO CLAVE)
V_oir = np.zeros((Ns, m))
Vfr1_oir = np.zeros((Ns, m))
Vfr2_oir = np.zeros((Ns, m))
Vfr3_oir = np.zeros((Ns, m))
PG_oir = np.zeros((Ns, m))  # Pérdidas y ganancias
PGfr1_oir = np.zeros((Ns, m))
PGfr2_oir = np.zeros((Ns, m))
PGfr3_oir = np.zeros((Ns, m))
PGT_oir = np.zeros((Ns, 1))
PGfr1T_oir = np.zeros((Ns, 1))
PGfr2T_oir = np.zeros((Ns, 1))
PGfr3T_oir = np.zeros((Ns, 1))

#N_oir
DeltaX_s_oir = DeltaX_port[:, int(sum(n_if[:3])) : int(sum(n_if[:4]))]  # Key step
x0_oir = X_oir[0, :]  # Key step

for i in range(Ns):
  X_s_oir[i, :] = x0_oir * np.exp(DeltaX_s_oir[i, :])

  V_oir[i, :] = opctint(X_s_oir[i, :(n_if[3][0]//3)], X_s_oir[i, (n_if[3][0]//3):(n_if[3][0]*2//3)],
                        K_oir, X_s_oir[i, (n_if[3][0]*2//3):], plazos_oir, cp_oir, cs_oir, pr_oir, dct_oir) * contratos_oir * nominal_oir

  Vfr1_oir[i, :] = opctint(X_s_oir[i, :(n_if[3][0]//3)], x0_oir[(n_if[3][0]//3):(n_if[3][0]*2//3)],
                           K_oir, x0_oir[(n_if[3][0]*2//3):], plazos_oir, cp_oir, cs_oir, pr_oir, dct_oir) * contratos_oir * nominal_oir

  Vfr2_oir[i, :] = opctint(x0_oir[:(n_if[3][0]//3)], X_s_oir[i, (n_if[3][0]//3):(n_if[3][0]*2//3)],
                           K_oir, x0_oir[(n_if[3][0]*2//3):], plazos_oir, cp_oir, cs_oir, pr_oir, dct_oir) * contratos_oir * nominal_oir

  Vfr3_oir[i, :] = opctint(x0_oir[:(n_if[3][0]//3)], x0_oir[(n_if[3][0]//3):(n_if[3][0]*2//3)],
                           K_oir, X_s_oir[i, (n_if[3][0]*2//3):], plazos_oir, cp_oir, cs_oir, pr_oir, dct_oir) * contratos_oir * nominal_oir

  PG_oir[i, :] = V_oir[i, :] - V0_oir
  PGfr1_oir[i, :] = Vfr1_oir[i, :] - V0_oir
  PGfr2_oir[i, :] = Vfr2_oir[i, :] - V0_oir
  PGfr3_oir[i, :] = Vfr3_oir[i, :] - V0_oir

  PGT_oir[i, 0] = np.sum(PG_oir[i, :])
  PGfr1T_oir[i, 0] = np.sum(PGfr1_oir[i, :])
  PGfr2T_oir[i, 0] = np.sum(PGfr2_oir[i, :])
  PGfr3T_oir[i, 0] = np.sum(PGfr3_oir[i, :])

#VaR por posición
VaRCont_oir=np.zeros((1, m))[0]
VaRfr1_oir=np.zeros((1, m))[0]
VaRfr2_oir=np.zeros((1, m))[0]
VaRfr3_oir=np.zeros((1, m))[0]
CVaRCont_oir=np.zeros((1, m))[0]
CVaRfr1_oir=np.zeros((1, m))[0]
CVaRfr2_oir=np.zeros((1, m))[0]
CVaRfr3_oir=np.zeros((1, m))[0]

for i in range(m):
  VaRCont_oir[i] = np.quantile(PG_oir[:, i], 1-alpha)
  VaRfr1_oir[i] = np.quantile(PGfr1_oir[:, i], 1-alpha)
  VaRfr2_oir[i] = np.quantile(PGfr2_oir[:, i], 1-alpha)
  VaRfr3_oir[i] = np.quantile(PGfr3_oir[:, i], 1-alpha)

  CVaRfr1_oir[i] = np.mean(np.column_stack((np.arange(0, Ns),PGfr1_oir[:, i]))[np.where(PGfr1_oir[:, i] < VaRfr1_oir[i])[0],1])
  CVaRfr2_oir[i] = np.mean(np.column_stack((np.arange(0, Ns),PGfr2_oir[:, i]))[np.where(PGfr2_oir[:, i] < VaRfr2_oir[i])[0],1])
  CVaRfr3_oir[i] = np.mean(np.column_stack((np.arange(0, Ns),PGfr3_oir[:, i]))[np.where(PGfr3_oir[:, i] < VaRfr3_oir[i])[0],1])
  CVaRCont_oir[i] = np.mean(np.column_stack((np.arange(0, Ns),PG_oir[:, i]))[np.where(PG_oir[:, i] < VaRCont_oir[i])[0],1])

VaRCont_oir
VaRfr1_oir
VaRfr2_oir
CVaRCont_oir
CVaRfr1_oir
CVaRfr2_oir

VaRTotal_oir = np.quantile(PGT_oir, 1-alpha)
VaRTotalfr1_oir = np.quantile(PGfr1T_oir, 1-alpha)
VaRTotalfr2_oir = np.quantile(PGfr2T_oir, 1-alpha)
VaRTotalfr3_oir = np.quantile(PGfr3T_oir, 1-alpha)

CVaRTotal_oir = np.mean(PGT_oir[np.where(PGT_oir < VaRTotal_oir)])
CVaRTotalfr1_oir = np.mean(PGfr1T_oir[np.where(PGfr1T_oir < VaRTotalfr1_oir)])
CVaRTotalfr2_oir = np.mean(PGfr2T_oir[np.where(PGfr2T_oir < VaRTotalfr2_oir)])
CVaRTotalfr3_oir = np.mean(PGfr3T_oir[np.where(PGfr3T_oir < VaRTotalfr3_oir)])

print("VaRCont_oir:", VaRCont_oir)
print("VaRfr1_oir:", VaRfr1_oir)
print("VaRfr2_oir:", VaRfr2_oir)
print("CVaRCont_oir:", CVaRCont_oir)
print("CVaRfr1_oir:", CVaRfr1_oir)
print("CVaRfr2_oir:", CVaRfr2_oir)

print("VaR Total:", VaRTotal_oir)
print("VaR Total fr1:", VaRTotalfr1_oir)
print("VaR Total fr2:", VaRTotalfr2_oir)
print("VaR Total fr3:", VaRTotalfr3_oir)
print("CVaR Total:", CVaRTotal_oir)
print("CVaR Total fr1:", CVaRTotalfr1_oir)
print("CVaR Total fr2:", CVaRTotalfr2_oir)
print("CVaR Total fr3:", CVaRTotalfr3_oir)

<ipython-input-33-950364892ffb>:17: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  DeltaX_s_oir = DeltaX_port[:, int(sum(n_if[:3])) : int(sum(n_if[:4]))]  # Key step


VaRCont_oir: [-0.18821507 -0.24676105 -0.2284084  -0.0366196  -0.37983541]
VaRfr1_oir: [-3.22923941e-03 -6.24926876e-03 -9.35222072e-04 -9.42690643e-05
 -7.70629163e-04]
VaRfr2_oir: [-0.12482202 -0.15741204 -0.14056631 -0.01153866 -0.22718078]
CVaRCont_oir: [-0.29679146 -0.30496454 -0.31390285 -0.09292922 -0.55841926]
CVaRfr1_oir: [-0.00472425 -0.00958835 -0.00178691 -0.00015315 -0.0015457 ]
CVaRfr2_oir: [-0.14295932 -0.20039256 -0.18185362 -0.02060295 -0.29777274]
VaR Total: -0.5367370301837612
VaR Total fr1: -0.004062002841645053
VaR Total fr2: -0.24882721733298174
VaR Total fr3: -0.4949123086695819
CVaR Total: -0.8011669524317732
CVaR Total fr1: -0.006063776689294913
CVaR Total fr2: -0.3180421302102308
CVaR Total fr3: -0.9398928772247599


## Forward TDC

In [ ]:
m = len(plazos_fwd) # Número de columnas en plazos_fwd (PASO CLAVE)
X_s_fwtdc = np.zeros((Ns, n_if[4][0]))  # Factores de riesgo simulados (PASO CLAVE)
V_fwtdc = np.zeros((Ns, m))
Vfr1_fwtdc = np.zeros((Ns, m))
Vfr2_fwtdc = np.zeros((Ns, m))
Vfr3_fwtdc = np.zeros((Ns, m))
PG_fwtdc = np.zeros((Ns, m))  # Pérdidas y ganancias
PGfr1_fwtdc = np.zeros((Ns, m))
PGfr2_fwtdc = np.zeros((Ns, m))
PGfr3_fwtdc = np.zeros((Ns, m))

PGT_fwtdc = np.zeros((Ns, 1))
PGfr1T_fwtdc = np.zeros((Ns, 1))
PGfr2T_fwtdc = np.zeros((Ns, 1))
PGfr3T_fwtdc = np.zeros((Ns, 1))

#N_fwtdc
DeltaX_s_fwtdc = DeltaX_port[:, int(sum(n_if[:4])) : int(sum(n_if[:5]))]  # Key step
x0_fwtdc = X_futtdc[0, :]  # Key step

for i in range(Ns):
    X_s_fwtdc[i, :] = x0_fwtdc * np.exp(DeltaX_s_fwtdc[i, :])

    V_fwtdc[i, :] = futuroTC(plazos_fwd, X_s_fwtdc[i, :((n_if[4][0]-1)//2)], X_s_fwtdc[i, ((n_if[4][0]-1)//2+1):(n_if[4][0])], X_s_fwtdc[i, (n_if[4][0]-1)], kst_fwd) * contratos_fwd * nominal_fwd

    Vfr1_fwtdc[i, :] = futuroTC(plazos_fwd, X_s_fwtdc[i, :((n_if[4][0]-1)//2)], x0_fwtdc[((n_if[4][0]-1)//2+1):(n_if[4][0])], x0_fwtdc[(n_if[4][0]-1)], kst_fwd) * contratos_fwd * nominal_fwd

    Vfr2_fwtdc[i, :] = futuroTC(plazos_fwd, x0_fwtdc[:((n_if[4][0]-1)//2)], X_s_fwtdc[i, ((n_if[4][0]-1)//2+1):(n_if[4][0])], x0_fwtdc[(n_if[4][0]-1)], kst_fwd) * contratos_fwd * nominal_fwd
    Vfr3_fwtdc[i, :] = futuroTC(plazos_fwd, x0_fwtdc[:((n_if[4][0]-1)//2)], x0_fwtdc[((n_if[4][0]-1)//2+1):(n_if[4][0])], X_s_fwtdc[i, (n_if[4][0]-1)], kst_fwd) * contratos_fwd * nominal_fwd

    PG_fwtdc[i, :] = V_fwtdc[i, :] - V0_fwtdc
    PGfr1_fwtdc[i, :] = Vfr1_fwtdc[i, :] - V0_fwtdc
    PGfr2_fwtdc[i, :] = Vfr2_fwtdc[i, :] - V0_fwtdc
    PGfr3_fwtdc[i, :] = Vfr3_fwtdc[i, :] - V0_fwtdc

    PGT_fwtdc[i, 0] = np.sum(PG_fwtdc[i, :])
    PGfr1T_fwtdc[i, 0] = np.sum(PGfr1_fwtdc[i, :])
    PGfr2T_fwtdc[i, 0] = np.sum(PGfr2_fwtdc[i, :])
    PGfr3T_fwtdc[i, 0] = np.sum(PGfr3_fwtdc[i, :])

#VaR por posición
VaRCont_fwtdc=np.zeros((1, m))[0]
VaRfr1_fwtdc=np.zeros((1, m))[0]
VaRfr2_fwtdc=np.zeros((1, m))[0]
VaRfr3_fwtdc=np.zeros((1, m))[0]
CVaRCont_fwtdc=np.zeros((1, m))[0]
CVaRfr1_fwtdc=np.zeros((1, m))[0]
CVaRfr2_fwtdc=np.zeros((1, m))[0]
CVaRfr3_fwtdc=np.zeros((1, m))[0]

for i in range(m):
  VaRCont_fwtdc[i]=np.quantile(PG_fwtdc[:, i], 1-alpha)
  VaRfr1_fwtdc[i]=np.quantile(PGfr1_fwtdc[:, i], 1-alpha)
  VaRfr2_fwtdc[i]=np.quantile(PGfr2_fwtdc[:, i], 1-alpha)
  VaRfr3_fwtdc[i]=np.quantile(PGfr3_fwtdc[:, i], 1-alpha)

  CVaRfr1_fwtdc[i]= np.mean(np.column_stack((np.arange(0, Ns),PGfr1_fwtdc[:, i]))[np.where(PGfr1_fwtdc[:, i] < VaRfr1_fwtdc[i])[0],1])
  CVaRfr2_fwtdc[i]= np.mean(np.column_stack((np.arange(0, Ns),PGfr2_fwtdc[:, i]))[np.where(PGfr2_fwtdc[:, i] < VaRfr2_fwtdc[i])[0],1])
  CVaRfr3_fwtdc[i]= np.mean(np.column_stack((np.arange(0, Ns),PGfr3_fwtdc[:, i]))[np.where(PGfr3_fwtdc[:, i] < VaRfr3_fwtdc[i])[0],1])
  CVaRCont_fwtdc[i]= np.mean(np.column_stack((np.arange(0, Ns),PG_fwtdc[:, i]))[np.where(PG_fwtdc[:, i] < VaRCont_fwtdc[i])[0],1])

VaRCont_fwtdc
VaRfr1_fwtdc
VaRfr2_fwtdc
VaRfr3_fwtdc
CVaRCont_fwtdc
CVaRfr1_fwtdc
CVaRfr2_fwtdc
CVaRfr3_fwtdc


#VaR Total
VaRTotal_fwtdc = np.quantile(PGT_fwtdc, 1-alpha)
VaRTotalfr1_fwtdc = np.quantile(PGfr1T_fwtdc, 1-alpha)
VaRTotalfr2_fwtdc = np.quantile(PGfr2T_fwtdc, 1-alpha)
VaRTotalfr3_fwtdc = np.quantile(PGfr3T_fwtdc, 1-alpha)

CVaRTotal_fwtdc = np.mean(PGT_fwtdc[np.where(PGT_fwtdc < VaRTotal_fwtdc)])
CVaRTotalfr1_fwtdc = np.mean(PGfr1T_fwtdc[np.where(PGfr1T_fwtdc < VaRTotalfr1_fwtdc)])
CVaRTotalfr2_fwtdc = np.mean(PGfr2T_fwtdc[np.where(PGfr2T_fwtdc < VaRTotalfr2_fwtdc)])
CVaRTotalfr3_fwtdc = np.mean(PGfr3T_fwtdc[np.where(PGfr2T_fwtdc < VaRTotalfr2_fwtdc)])

print(VaRTotal_fwtdc)
print(VaRTotalfr1_fwtdc)
print(VaRTotalfr2_fwtdc)
print(VaRTotalfr3_fwtdc)
print(CVaRTotal_fwtdc)
print(CVaRTotalfr1_fwtdc)
print(CVaRTotalfr2_fwtdc)
print(CVaRTotalfr3_fwtdc)

7564.455540456817
7679.821253361952
7673.1241119542765
7576.6767836451345
7554.438257512803
7675.041312710891
7671.897617586019
7588.276575121188


<ipython-input-31-a9e7f8b9d4a9>:18: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  DeltaX_s_fwtdc = DeltaX_port[:, int(sum(n_if[:4])) : int(sum(n_if[:5]))]  # Key step


In [ ]:
n_if[5][0]

7

## Forwards Indices

In [ ]:
#Cálculo de matriz de pérdidas y ganancias FUTUROS IPC
m = len(plazos_fwd_ind)  # Número de columnas en plazos_fwd_ind (PASO CLAVE)
X_s_fwind = np.zeros((Ns, n_if[5][0]))  # Factores de riesgo simulados (PASO CLAVE)
V_fwind = np.zeros((Ns, m))
Vfr1_fwind = np.zeros((Ns, m))
Vfr2_fwind = np.zeros((Ns, m))
Vfr3_fwind = np.zeros((Ns, m))
PG_fwind = np.zeros((Ns, m))  # Pérdidas y ganancias
PGfr1_fwind = np.zeros((Ns, m))
PGfr2_fwind = np.zeros((Ns, m))
PGfr3_fwind = np.zeros((Ns, m))

PGT_fwind = np.zeros((Ns, 1))
PGfr1T_fwind = np.zeros((Ns, 1))
PGfr2T_fwind = np.zeros((Ns, 1))
PGfr3T_fwind = np.zeros((Ns, 1))

DeltaX_s_fwind=DeltaX_port[:, int(sum(n_if[:5])) : int(sum(n_if[:6]))]  # Key step
x0_fwind=X_futind[0,:] # Key step

for i in range(Ns):
  X_s_fwind[i, :] = x0_fwind * np.exp(DeltaX_s_fwind[i, :])
#No coinciden los valores pues son 2,2,3
  plazos_fwd_ind_1 = X_s_fwind[i, :(n_if[5][0]-1)//2]#0-2
  plazos_fwd_ind_2 = X_s_fwind[i, ((n_if[5][0]-1)//2+1):(n_if[5][0])]#2-4
  plazos_fwd_ind_3 = X_s_fwind[i, (n_if[5][0]-1)]#4-7
  # X_s_fwtdc[i, :((n_if[4][0]-1)//2)], X_s_fwtdc[i, ((n_if[4][0]-1)//2+1):(n_if[4][0])], X_s_fwtdc[i, (n_if[4][0]-1)],
  V_fwind[i, :] = futuroTC(plazos_fwd_ind, plazos_fwd_ind_1, plazos_fwd_ind_2, plazos_fwd_ind_3, kst_fwd_ind) * contratos_fwd_ind * nominal_fwd_ind
  # V0_fwind = futuroTC(plazos_fwd_ind, X1_fwind[0, :], X2_fwind[0, :], X3_find[0, :], kst_fwd_ind) * contratos_fwd_ind * nominal_fwd_ind
  Vfr1_fwind[i, :] = V_fwind[i, :]
  Vfr2_fwind[i, :] = V_fwind[i, :]
  Vfr3_fwind[i, :] = V_fwind[i, :]

  PG_fwind[i, :] = V_fwind[i, :] - V0_fwind
  PGfr1_fwind[i, :] = Vfr1_fwind[i, :] - V0_fwind
  PGfr2_fwind[i, :] = Vfr2_fwind[i, :] - V0_fwind
  PGfr3_fwind[i, :] = Vfr3_fwind[i, :] - V0_fwind

  PGT_fwind[i, 0] = np.sum(PG_fwind[i, :])
  PGfr1T_fwind[i, 0] = np.sum(PGfr1_fwind[i, :])
  PGfr2T_fwind[i, 0] = np.sum(PGfr2_fwind[i, :])
  PGfr3T_fwind[i, 0] = np.sum(PGfr3_fwind[i, :])

#VaR por posición
VaRCont_fwind=np.zeros((1, m))[0]
VaRfr1_fwind=np.zeros((1, m))[0]
VaRfr2_fwind=np.zeros((1, m))[0]
VaRfr3_fwind=np.zeros((1, m))[0]
CVaRCont_fwind=np.zeros((1, m))[0]
CVaRfr1_fwind=np.zeros((1, m))[0]
CVaRfr2_fwind=np.zeros((1, m))[0]
CVaRfr3_fwind=np.zeros((1, m))[0]

for i in range(m):
  VaRCont_fwind[i]=np.quantile(PG_fwind[:, i], 1-alpha)
  VaRfr1_fwind[i]=np.quantile(PGfr1_fwind[:, i], 1-alpha)
  VaRfr2_fwind[i]=np.quantile(PGfr2_fwind[:, i], 1-alpha)
  VaRfr3_fwind[i]=np.quantile(PGfr3_fwind[:, i], 1-alpha)

  CVaRfr1_fwind[i]= np.mean(np.column_stack((np.arange(0, Ns),PGfr1_fwind[:, i]))[np.where(PGfr1_fwind[:, i] < VaRfr1_fwind[i])[0],1])
  CVaRfr2_fwind[i]= np.mean(np.column_stack((np.arange(0, Ns),PGfr2_fwind[:, i]))[np.where(PGfr2_fwind[:, i] < VaRfr2_fwind[i])[0],1])
  CVaRfr3_fwind[i]= np.mean(np.column_stack((np.arange(0, Ns),PGfr3_fwind[:, i]))[np.where(PGfr3_fwind[:, i] < VaRfr3_fwind[i])[0],1])
  CVaRCont_fwind[i]= np.mean(np.column_stack((np.arange(0, Ns),PG_fwind[:, i]))[np.where(PG_fwind[:, i] < VaRCont_fwind[i])[0],1])

VaRCont_fwind
VaRfr1_fwind
VaRfr2_fwind
CVaRCont_fwind
CVaRfr1_fwind
CVaRfr2_fwind

#VaR Total

VaRTotal_fwind=np.quantile(PGT_fwind, 1-alpha)
VaRTotalfr1_fwind=np.quantile(PGfr1T_fwind, 1-alpha)
VaRTotalfr2_fwind=np.quantile(PGfr2T_fwind, 1-alpha)
VaRTotalfr3_fwind=np.quantile(PGfr3T_fwind, 1-alpha)

CVaRTotal_fwind= np.mean(PGT_fwind[np.where(PGT_fwind < VaRTotal_fwind)])
CVaRTotalfr1_fwind= np.mean(PGfr1T_fwind[np.where(PGfr1T_fwind < PGfr1T_fwind)])
CVaRTotalfr2_fwind= np.mean(PGfr2T_fwind[np.where(PGfr2T_fwind < PGfr2T_fwind)])
CVaRTotalfr3_fwind= np.mean(PGfr3T_fwind[np.where(PGfr3T_fwind < PGfr3T_fwind)])

print(VaRTotal_fwind)
print(VaRTotalfr1_fwind)
print(VaRTotalfr2_fwind)
print(VaRTotalfr3_fwind)
print(CVaRTotal_fwind)
print(CVaRTotalfr1_fwind)
print(CVaRTotalfr2_fwind)
print(CVaRTotalfr3_fwind)

#VaR Total

VaRTotal_fwind=np.quantile(PGT_fwind, 1-alpha)
VaRTotalfr1_fwind=np.quantile(PGfr1T_fwind, 1-alpha)
VaRTotalfr2_fwind=np.quantile(PGfr2T_fwind, 1-alpha)
VaRTotalfr3_fwind=np.quantile(PGfr3T_fwind, 1-alpha)

CVaRTotal_fwind= np.mean(PGT_fwind[np.where(PGT_fwind < VaRTotal_fwind)])
CVaRTotalfr1_fwind= np.mean(PGfr1T_fwind[np.where(PGfr1T_fwind < PGfr1T_fwind)])
CVaRTotalfr2_fwind= np.mean(PGfr2T_fwind[np.where(PGfr2T_fwind < PGfr2T_fwind)])
CVaRTotalfr3_fwind= np.mean(PGfr3T_fwind[np.where(PGfr3T_fwind < PGfr3T_fwind)])

print(VaRTotal_fwind)
print(VaRTotalfr1_fwind)
print(VaRTotalfr2_fwind)
print(VaRTotalfr3_fwind)
print(CVaRTotal_fwind)
print(CVaRTotalfr1_fwind)
print(CVaRTotalfr2_fwind)
print(CVaRTotalfr3_fwind)

137384.7686498867
137384.7686498867
137384.7686498867
137384.7686498867
136175.28088338076
nan
nan
nan
137384.7686498867
137384.7686498867
137384.7686498867
137384.7686498867
136175.28088338076
nan
nan
nan


<ipython-input-30-4fcc48e6f919>:18: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  DeltaX_s_fwind=DeltaX_port[:, int(sum(n_if[:5])) : int(sum(n_if[:6]))]  # Key step


In [ ]:
X_s_fwind[i, (n_if[5][0]-1)]

50203.97996540688

# Calculo de VaR del portafolio

In [ ]:
# Acciones
PGPort_ACC = PGfr2_acc_div #+ PGfr3T_fwind
VaRPort_ACC = np.quantile(PGPort_ACC, 1-alpha)
CVaRPort_ACC = np.mean(PGPort_ACC[PGPort_ACC < VaRPort_ACC])

# Tasa de Interés
PGPort_TI = PGT_bd + PGT_sw + PGfr1T_oir + PGfr2T_oir +  PGfr1T_fwtdc + PGfr2T_fwtdc #+ PGfr1T_fwind + PGfr2T_fwind
VaRPort_TI = np.quantile(PGPort_TI, 1-alpha)
CVaRPort_TI = np.mean(PGPort_TI[PGPort_TI < VaRPort_TI])

# Tipo de cambio
PGPort_TDC = PGfr1_acc_div + PGfr3T_fwtdc
VaRPort_TDC = np.quantile(PGPort_TDC, 1-alpha)
CVaRPort_TDC = np.mean(PGPort_TDC[PGPort_TDC < VaRPort_TDC])

# Volatilidad
PGPort_VOL = PGfr3T_oir
VaRPort_VOL = np.quantile(PGPort_VOL, 1-alpha)
CVaRPort_VOL = np.mean(PGPort_VOL[PGPort_VOL < VaRPort_VOL])

# Riesgo de todo el portafolio
PGT_Port = np.column_stack((PGPort_ACC, PGPort_TI, PGPort_TDC, PGPort_VOL))
VaRTotal_Port = np.quantile(PGT_Port, 1-alpha)
CVaRTotal_Port = np.mean(PGT_Port[PGT_Port < VaRTotal_Port])

print(VaRTotal_Port)
print(CVaRTotal_Port)
print(V0T_port)

-20582.192032581916
-34232.210281856846
981899.493300085
